# 텍스트 데이터 다루기

### 목표

1. 분산 표현에 대한 직관적 이해 얻기
2. 문자 데이터를 정제하는 방법 배우기
3. 토큰화의 여러 가지 기법 배우기
4. 단어 Embedding을 구축하는 방법에 대해 가볍게 맛보기

<hr>

### 자연 언어와 프로그래밍 언어

- 자연어와 달리 컴퓨터 프로그래밍을 위해 특별히 개발된 FORTRAN, PASCAL, C 등의 언어를 인공 언어 혹은 프로그래밍 언어라고 부른다.
- 자연어는 문맥 의존 문법(Context-sensitive Grammar), 프로그래밍 언어는 문맥 자유 문법(Context-free Grammar)를 따른다.
- 문맥 자유 문법 언어란 문맥과 상황에 따라 그 뜻이 달라지지 않는 언어, 다시 말해 문맥을 완전히 배제하고도 해석이 가능한 언어를 말한다.
- 기계적으로 언어를 해석하는 parser를 만들었을 때 그 언어가 문맥 자유 문법을 따르는 언어여야 해석상 의미의 모호성이 생기지 않는 파싱트리를 만들 수 있다.
- 예를 들어 "Alice drove down the street in her car." 라는 문장은 두가지로 해석될 수 있는 모호성을 가지고 있다.
    - 앨리스는 자기 차를 운전하여 거리를 달려갔다.
    - 앨리스는 자기 차 안에 있는 거리를 운전해 달려갔다.
- 문법적으로 가능한 위 두 가지 해석 중 어느 것이 맞는지를 파악하려면 real world knowledge, 즉 현실 세계에 대한 지식(예를 들어 '자동차 안에 거리가 있을 리는 없다')이 필요하다.
- 즉 자연어 처리 모델에는 단순히 언어의 문법만으로 충분하지 않고 그 언어의 의미까지 이해하는 과정이 필요하다.

### Khaiii 형태소 분석기 준비하기

카카오에서 제공하는 Khaiii 형태소 분석기 설치

> $ sudo apt install cmake   # Khaiii의 빌드 과정을 위해 cmake를 필요로 합니다.

> $ pip install torch     # Khaiii는 구동을 위해 파이토치를 필요로 합니다. 

> $ cd ~/aiffel/text_preprocess

> $ git clone https://github.com/modulabs/khaiii_wrapper.git

> $ cd ~/aiffel/text_preprocess/khaiii_wrapper/khaiii_pos

> $ ./install_khaiii.sh

<hr>

## 문자 데이터를 정제하는 방법 배우기

### 전처리: 자연어의 노이즈 제거

- 1) 불완전한 문장으로 구성된 대화의 경우
    - 한 문장씩 주고 받는 대화와 달리 메신저는 한 문장을 여러 번에 나눠 전송하거나 여러 문장을 한 번에 전송하는 경우
    - 예1) A: "아니아니" "오늘이 아니고" "내일이지" / B: "그럼 오늘 사야겠네. 내일 필요하니까?"

- 2) 문장의 길이가 너무 길거나 짧은 경우
    - 아주 짧은 문장은 의미가 없을 수 있고, 대체로 사용빈도가 높은 리액션에 해당하는 경우가 많아서 언어 모델을 왜곡시킬 우려가 있기 때문에 제외해 주는 게 좋다.
    - 예1) A: "ㅋㅋㅋ", "ㅠㅠㅠ"
    - 아주 긴 문장은 대화와는 관계가 없는 문장일 수 있다.
    - 예2) A: "이 편지는 영국에서부터 시작되어…"
    
- 3) 채팅 데이터에서 문장 시간 간격이 너무 긴 경우
    - 서로의 말의 텀이 짧으면 그것은 대화가 아니라 서로 할말만 하는 상태일 수 있다.
    - 예1) A: "겨울왕국2" / B: "보러가자" / A: "엊그제 봤는데 잼씀" / B: "오늘 저ㄴ 아니 ㅡㅡ"
    - 혹은 말의 텀이 너무 길다면 그것은 연속된 대화로 보기 어렵다.
    - 예2) A: "나 10만원만 빌려줄 수 있어?" / …… / B: "아 미안 지금 봤다 아직 필요해?"
    
- 4) 바람직하지 않은 문장의 사용
    - 욕설의 비율이나, 오타의 비율이 높은 문장은 자연어 모델 학습에 사용하지 않는 것이 좋습니다.

#### - 단순한 노이즈 처리해보기

- 유형 1. 문장 부호
- 유형 2. 대소문자
- 유형 3. 특수 문자

In [1]:
# 유형 1. 문장 부호
def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " "+p+" ")
    
    return sentence

sentence = "Hi, my name is John."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

Hi ,  my name is John . 


In [2]:
# 유형 2. 대소문자
sentence = "First, open the first chapter."

print(sentence.lower())

first, open the first chapter.


In [3]:
# 유형 3. 특수문자
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


In [4]:
# 위 세 가지 유형에 대한 처리를 함수로 정의
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
    # 노이즈 유형 (1) 문장부호 공백추가
    for p in punc:
        text = text.replace(p, " " + p + " ")

    # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()

    return text

print(cleaning_text(corpus, [".", ",", "!", "?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it

<hr>

## 분산 표현에 대한 직관적 이해 얻기

#### - 희소 표현

- 단어의 희소 표현과 분산 표현은 모두 단어를 벡터로 표현하겠다는 점에서는 동일하지만, 단어의 의미를 표현하는 접근 방식에 차이가 있다.
- 희소 표현(Sparse Representation) 방식은 벡터의 각 차원마다 단어의 특정 의미 속성을 대응시키는 방식이다.
    - 예를 들어 사람의 성별을 표현하는 남자와 여자라는 두 단어를 수로 표현하려면 남자: [-1], 여자: [1] 형태로 표현해볼 수 있다.
    - 다음으로 등장한 단어는 소년과 소녀입니다. 두 단어는 각각 어린 남자와 여자를 의미하니, 앞서 생성한 "성별" 이라는 속성에 "나이가 어리다" 라는 속성을 추가해야 한다.
    - "나이가 많다" 라는 속성을 가진 할아버지, 할머니는 자동적으로 위 속성의 반대 값으로 표현할 수 있다.
    - 각 속성값들이 정수가 아니라 실수라면 더 다양한 표현이 가능하다.

In [5]:
"""
{
    //     [성별, 연령]
    남자: [-1.0, 0.0], // 이를테면 0.0 이 "관계없음 또는 중립적" 을 의미할 수 있겠죠!
    여자: [1.0, 0.0],
    소년: [-1.0, -0.7],
    소녀: [1.0, -0.7],
    할머니: [1.0, 0.7],
    할아버지: [-1.0, 0.7],
    아저씨: [-1.0, 0.2],
    아줌마: [1.0, 0.2]
}
"""

"""
{
    //      [성별, 연령, 과일, 색깔]
    남자: [-1.0, 0.0, 0.0, 0.0],
    여자: [1.0, 0.0, 0.0, 0.0],
    사과: [0.0, 0.0, 1.0, 0.5],   // 빨갛게 잘 익은 사과
    바나나: [0.0, 0.0, 1.0, -0.5] // 노랗게 잘 익은 바나나
}
"""

'\n{\n    //      [성별, 연령, 과일, 색깔]\n    남자: [-1.0, 0.0, 0.0, 0.0],\n    여자: [1.0, 0.0, 0.0, 0.0],\n    사과: [0.0, 0.0, 1.0, 0.5],   // 빨갛게 잘 익은 사과\n    바나나: [0.0, 0.0, 1.0, -0.5] // 노랗게 잘 익은 바나나\n}\n'

- 이처럼 희소 표현 방식은 단어를 고차원 벡터로 변환하는 것이다.
- 속성의 종류가 늘어나고 워드 벡터의 차원이 따라서 늘어나면, 대부분의 차원이 0.0으로 채워진다.
- 즉 불필요한 메모리와 연산량이 낭비되며, 이러한 희소  표현의 워드 벡터끼리는 단어들 간의 의미적 유사도를 계산할 수 없다는 문제가 있다.

#### - 코사인 유사도(Cosine Similarity)

- 두 벡터간의 코사인 각도를 이용해 구할 수 있는 벡터의 유사도를 의미한다.
- 두 벡터의 방향이 완전히 동일하면 1, 직각을 이루면 0, 반대 방향은 -1의 값을 갖는다.
- 즉 1에 가까울수록 유사도가 높다고 할 수 있다.

![image](https://user-images.githubusercontent.com/80008411/134461551-57b7a4a3-eb0f-4996-9148-3857e67c8a63.png)

- 식으로 표현하면 다음과 같다.

![image](https://user-images.githubusercontent.com/80008411/134461664-fc7f228c-fe35-4e47-b429-e73415a77785.png)

In [6]:
# 쉽게 코사인 유사도 계산하기
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(a, b):
    return dot(a, b)/(norm(a) * norm(b))

In [7]:
word_1 = np.array([-1.0, 0.0, 0.0, 0.0])
word_2 = np.array([0.0, 0.0, 1.0, 0.5])

cos_sim(word_1, word_2)

0.0

공유하는 의미 속성이 없는 위 두 벡터의 내적은 0이 되므로, 코사인 유사도 역시 0이 된다.

즉 두 단어 사이에는 아무런 의미적 유사도가 없다.

#### - 분산 표현(Distributed Representation)

- 분산 표현 방식은 Embedding layer를 사용해 각 단어가 몇 차원의 속성을 가질지 정의한다.
- 만약 100개의 단어를 256 차원의 속성으로 표현하고 싶다면 embedding layer는 아래와 같이 정의된다.

> embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=256)

- 이 분산 표현에는 일일이 정의할 수 없는 추상적인 속성들이 256차원 안에 골고루 분산되어 표현된다.
- 희소 표현처럼 속성값을 임의로 지정하는 것이 아니라, 수많은 텍스트 데이터를 읽어가며 적합한 값을 찾아가는 것이다.
- 적절히 훈련된 분산 표현 모델을 통해 단어 간 의미 유사도를 계산하거나, 이를 feature로 삼아 복잡한 자연어 처리 모델을 훈련시킬 수 있다.

<hr>

## 토큰화의 여러 가지 기법 배우기

- 문장을 어떤 기준으로 쪼개었을 때, 쪼개진 각 단어들을 토큰이라고 한다.
- 그리고 그 쪼개진 기준이 토큰화 기법이라고 할 수 있다.

### 1) 공백 기반 토큰화

- 간단히 split() 함수를 이용한다.

In [8]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print("문장이 포함하는 Tokens:", tokens)

문장이 포함하는 Tokens: ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attenti

### 2) 형태소 기반 토큰화

- 형태소란 뜻을 가진 가장 작은 말의 단위
- 한국어 형태소 분석기
    - [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)
    - [kakao/khaiii](https://github.com/kakao/khaiii) # 윈도우 지원 안 함
- [블로그: 한국어 형태소 분석기 성능 비교](https://iostream.tistory.com/144)
    - 속도 측면에서 가장 뛰어난 분석기: mecab
    - 시간은 오래 걸리지만 정확도가 높은 분석기: KOMORAN (+mecab, 꼬꼬마)

In [9]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt
import khaiii

api = khaiii.KhaiiiApi()
api.open()

In [10]:
# Khaiii를 konlpy tokenizer처럼 사용하기 위한 wrapper class입니다. 

class Khaiii():
    def pos(self, phrase, flatten=True, join=False):
        """POS tagger.

        :param flatten: If False, preserves eojeols.
        :param join: If True, returns joined sets of morph and tag.

        """
        sentences = phrase.split('\n')
        morphemes = []
        if not sentences:
            return morphemes

        for sentence in sentences:
            for word in api.analyze(sentence):
                result = [(m.lex, m.tag) for m in word.morphs]
                if join:
                    result = ['{}/{}'.format(m.lex, m.tag) for m in word.morphs]

                morphemes.append(result)

        if flatten:
            return sum(morphemes, [])

        return morphemes

In [11]:
tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt(),Khaiii()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))
    print('--------------\n')

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]
--------------

[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
--------------

[Komoran] 
[('코로나바이러스', 'N

#### - 사전에 없는 단어의 문제

- 공백 기반, 형태소 기반 토큰화 기법들 모두 의미를 가지는 단위로 토큰을 생성한다.
- 둘 다 데이터에 포함되는 모든 단어를 처리할 수 없기 때문에 자주 등장한 상위 N개의 단어만 사용하고 나머지는 \<unk> 같은 특수 토큰으로 치환한다.
- 이렇게 새로 등장한 단어들에 대해 발생하는 OOV(Out-Of-Vocabulary) 문제를 해결하기 위해 Wordpiece Model이 등장했다.

<hr>

## 단어 Embedding을 구축하는 방법에 대해 가볍게 맛보기

### Wordpiece Model(WPM)

- 한 단어를 여러 개의 Subword의 집합으로 보는 방법
- 예를 들어 preview와 predict를 접두어인 pre와 연계해서, pre+view, pre+dict로 학습하는 것이다.
- 이를 이해하기 전에 먼저 Byte Pair Encoding(BPE)에 대해 알아보자.

#### - BPE(Byte Pair Encoding)

- 원래는 1994년 데이터 압축을 위해 생겨났다.
- 데이터에서 많이 등장하는 바이트 쌍을 새로운 단어로 치환해 압축하는 작업을 반복하는 방식이다.
- 이를 토큰화에 적용하자고 제안한 것은 2015년이었다.
- 모든 단어를 바이트들의 집합으로 취급해 자주 등장하는 바이트 쌍을 합치면, 접두어나 접미어의 의미를 캐치할 수 있고, 처음 당장하는 단어는 문자들의 조합으로 나타내어 OOV 문제를 완전히 해결할 수 있다.
- [관련 논문](https://arxiv.org/pdf/1508.07909.pdf)

In [12]:
# 예시
"""
aaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.
→ 
ZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.
→ 
ZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면
Y=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.
→ 
XdXac
Z=aa
Y=ab
X=ZY       # 압축이 완료되었습니다!
"""

'\naaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.\n→ \nZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.\nZ=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.\n→ \nZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.\nZ=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면\nY=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.\n→ \nXdXac\nZ=aa\nY=ab\nX=ZY       # 압축이 완료되었습니다!\n'

In [13]:
# 논문의 예시 코드
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merge Vocab:", token_vocab)

>> Step 1
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3} 

>> Step 2
다음 문자 쌍을 치환: est
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 3
다음 문자 쌍을 치환: lo
변환된 Vocab:
 {'lo w ': 5, 'lo w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 4
다음 문자 쌍을 치환: low
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 5
다음 문자 쌍을 치환: ne
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'ne w est ': 6, 'w i d est ': 3} 

Merge Vocab: ['es', 'est', 'lo', 'low', 'ne']


#### - WPM

- BPE를 변형해 구글에서 제안한 알고리즘
    - 차별성 1) 공백 복원을 위해 단어의 시작 부분에 언더바를 추가한다.
    - 차별성 2) 빈도수가 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합친다.

- 소스코드가 공개되어 있지는 않지만, 대신 구글의 SentencePiece 라이브러리를 통해 고성능의 BPE를 사용할 수 있다. (전처리 과정도 포함되어 있음)

- 또 다른 문제: 아직 완벽하게 단어의 분산 표현을 얻지 못한다. 예를 들어 자동차를 \_자동, \_차 로 분리한다면 속성이 아무리 추상적이어도 마시는 차인지 달리는 차인지 구분할 수 없기 때문

## 토큰에게 의미 부여하기

- 각 토큰들에 랜덤하게 실수를 부여하지 않도록, 즉 그들끼리도 유사도 연산을 할 수 있게 의미를 부여하는 방법이 있다.

- 1) Word2Vec
    - 동시에 등장하는 단어끼리 연관성이 있다는 아이디어로 시작된 알고리즘
    - (1) CBOW(Continuous Bag of Words)
        - 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법
        - 중심 단어를 center word, 예측에 사용되는 단어들을 주변 단어 context word 라고 한다.
        - 중심 단어를 예측하기 위해서 앞, 뒤로 몇 개의 단어를 볼지를 결정했다면 이 범위를 window 라고 한다.
        - 예를 들어 window가 2이고, 중심 단어가 sat이라면, 앞의 두 단어인 fat, cat, 그리고 뒤의 두 단어인 on, the를 참고한다.
        - 윈도우 크기를 정했다면, 위치를 움직여 주변 단어와 중심 단어를 바꿔가며 학습을 하기 위한 데이터셋을 만들 수 있다. 이를 sliding window라고 한다.

![image](https://user-images.githubusercontent.com/80008411/134473999-1e711713-57af-4771-94e1-bc683a5f6f65.png)

- 
    - (2) Skip-Gram
    - 중간에 있는 단어로 주변 단어들을 예측하는 방법
    
![image](https://user-images.githubusercontent.com/80008411/134474611-022caf9c-820c-4931-a510-9e8e837238e7.png)

- 2) FastText
    - Word2Vec은 자주 등장하는 단어와 그렇지 않은 단어의 연산이 크게 차이가 난다.
    - 또 한국어와 같은 교착어의 경우 동일한 어휘가 문맥 속에서 복잡한 문법적 규칙에 의해 다양하게 변한다. 이 경우 의미상 유사하지만 문법적으로 조금만 다른 어휘들이 모두 서로 무관한 개별 어휘로 처리되어 제대로 학습이 되지 않는 문제가 있다.
    - 자주 등장하지 않는 단어는 최악의 경우 단 한 번의 연산만을 거쳐 랜덤하게 초기화된 값과 크게 다르지 않은 상태로 알고리즘이 종료될 수 있다.
    - 특히 새로 등장한 단어에 대해 임베딩을 어떻게 구성해야 하는지를 해결하기 위해 등장한 것이 FastText이다.
    - 즉 단어를 n-gram의 subword level로 쪼개서 이 n-gram에 할당된 임베딩의 평균값을 사용하는 것이다.

- 3) ELMo
    - 위 임베딩 기법들의 고질적 문제는 바로 고정적이라는 것이다.
    - 즉 동음이의어를 처리할 수 없다.
    - Contextualized Word Embedding은 특정 단어뿐 아니라 그 주변 단어 배치의 맥락을 함께 고려한다.
    - ELMo는 처음 등장한 Contextualized Word Embedding 모델이다.
    - 양방향 LSTM을 이용하며, 순방향과 역방향 LSTM은 각각 서로의 hidden state를 공유하지 않고 학습할 때 로그 우도를 동시에 최대화하는 방식으로 학습된다.
    
![image](https://user-images.githubusercontent.com/80008411/134514704-bfe9b4f4-14a8-4c66-aa5e-c75bcbbaba77.png)    